In [4]:
import cdsapi
import xarray as xr
import pygrib
import numpy as np
import pandas as pd

import requests
import os
import xarray as xr
#What a permaculture project need to know when they start
import time
from geopy.geocoders import Nominatim

# select the years you want to download:
start_year = 1992
end_year = 2022
year_range = [i for i in range(start_year, end_year + 1)]
print(year_range)


c = cdsapi.Client()
try:
    data = c.retrieve("reanalysis-era5-single-levels-monthly-means",
    {"format": "grib",
     "product_type": "monthly_averaged_reanalysis_by_hour_of_day",
     "variable": ['10m_u_component_of_wind', '10m_v_component_of_wind', 
                '2m_temperature',
                'snowfall', 
                'soil_temperature_level_1', 
                'soil_type',
                'total_cloud_cover', 
                'total_precipitation',
                ],
    "area": [72,-11, 33, 40],  # North, West, South, East. 
    "year": year_range,
    "month": ['01', '02', '03',
           '04', '05', '06',
           '07', '08', '09',
           '10', '11', '12'],
    "time": ["00:00","01:00","02:00","03:00","04:00","05:00",
             "06:00","07:00","08:00","09:00","10:00","11:00",
             "12:00", "13:00","14:00","15:00","16:00","17:00",
             "18:00","19:00","20:00","21:00","22:00","23:00"]
    })

    # Get the location of the file to download
    url = data.location

    # Download the file
    response = requests.get(url)

    # Check if the request was successful
    response.raise_for_status()

except requests.exceptions.HTTPError as errh:
    print ("HTTP Error:",errh)
except requests.exceptions.ConnectionError as errc:
    print ("Error Connecting:",errc)
except requests.exceptions.Timeout as errt:
    print ("Timeout Error:",errt)
except requests.exceptions.RequestException as err:
    print ("Something went wrong with the request:",err)

else:
    # If the request was successful, write the file
    filename = 'my_app/data/download.grib'
    with open(filename, 'wb') as f:
        f.write(response.content)

    # Print the location where the file is saved


[1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]


2023-12-04 14:55:46,920 INFO Welcome to the CDS
2023-12-04 14:55:46,920 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels-monthly-means
2023-12-04 14:55:46,967 INFO Request is queued
2023-12-04 15:10:06,251 INFO Request is running
2023-12-04 16:14:12,439 INFO Request is completed
2023-12-04 16:14:12,491 WARNING DELETE https://cds.climate.copernicus.eu/api/v2/tasks/1e8a120e-9d08-4ec8-bae8-a11b2cc247ff returns 404 


In [3]:
print(f"File saved at: {os.path.abspath(filename)}")

File saved at: /Users/giacomo/Documents/projects/permaculture-climate/download.grib


In [30]:
filename = "my_app/data/download.grib"

# List of variables to load
variables = ['2t','10v','10u','tp','tcc']

# Dictionary to hold the datasets
datasets = {}
lat = 40
lon = -2
# Open the GRIB file for each variable using the short name parameter
for var in variables:
    ds = xr.open_dataset(filename, engine='cfgrib', 
                         backend_kwargs={'filter_by_keys': {'shortName': var}})
    print(ds)
    ds = ds.sel(latitude=slice(lat + 1, lat - 1), longitude=slice(lon - 1, lon + 1))
    datasets[var] = ds


<xarray.Dataset>
Dimensions:     (time: 8928, latitude: 157, longitude: 205)
Coordinates:
    number      int64 ...
  * time        (time) datetime64[ns] 1992-01-01 ... 2022-12-01T23:00:00
    step        timedelta64[ns] ...
    surface     float64 ...
  * latitude    (latitude) float64 72.0 71.75 71.5 71.25 ... 33.5 33.25 33.0
  * longitude   (longitude) float64 -11.0 -10.75 -10.5 ... 39.5 39.75 40.0
    valid_time  (time) datetime64[ns] ...
Data variables:
    t2m         (time, latitude, longitude) float32 ...
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2023-12-06T14:11 GRIB to CDM+CF via cfgrib-0.9.1...
<xarray.Dataset>
Dimensions:     (time: 8928, latitude: 157, longitude: 205)
Coordinates:
    nu

In [31]:
datasets['tp']

<xarray.Dataset>
Dimensions:     (time: 1116, step: 12, latitude: 9, longitude: 9)
Coordinates:
    number      int64 ...
  * time        (time) datetime64[ns] 1991-12-31T18:00:00 ... 2022-12-01T18:0...
  * step        (step) timedelta64[ns] 01:00:00 02:00:00 ... 11:00:00 12:00:00
    surface     float64 ...
  * latitude    (latitude) float64 41.0 40.75 40.5 40.25 ... 39.5 39.25 39.0
  * longitude   (longitude) float64 -3.0 -2.75 -2.5 -2.25 ... -1.5 -1.25 -1.0
    valid_time  (time, step) datetime64[ns] ...
Data variables:
    tp          (time, step, latitude, longitude) float32 ...
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2023-12-06T14:11 GRIB to CDM+CF via cfgrib-0.9.1...